In [1]:
import os
import getpass


os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Add tools later

In [1]:
#from tools.design_retriever import DesignRetrieverTool
from chains.design_rag import DesignRAG

# Initialize DesignRAG and create the tool
design_rag = DesignRAG()
#design_retriever = DesignRetrieverTool(rag=design_rag)

test_requirements = {
    "style_description": "Modern minimalist design with clean lines",
    "key_elements": ["clean typography", "white space", "grid layout"],
    "color_scheme": "Monochromatic with blue accents",
    "layout_preferences": "Grid-based with clear hierarchy",
    "mood": "Professional and sophisticated"
}

# Test the retriever
async def test_rag():
    print("Testing RAG retriever with requirements:")
    print("----------------------------------------")
    for key, value in test_requirements.items():
        print(f"{key}: {value}")
    print("\nRetrieved Designs:")
    print("----------------------------------------")
    
    results = await design_rag.query_similar_designs(test_requirements)
    print(results)

# Run the test
await test_rag()


Loaded 82 design documents
Testing RAG retriever with requirements:
----------------------------------------
style_description: Modern minimalist design with clean lines
key_elements: ['clean typography', 'white space', 'grid layout']
color_scheme: Monochromatic with blue accents
layout_preferences: Grid-based with clear hierarchy
mood: Professional and sophisticated

Retrieved Designs:
----------------------------------------
Generated query: Modern minimalist design with clean lines AND clean typography AND white space AND grid layout AND Monochromatic with blue accents AND Grid-based layout AND Professional AND sophisticated mood


/Users/owner/Desktop/Projects/ai-maker-space/code/ImagineUI/src/chains/design_rag.py:114: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(


Design 003:
Design 003:
Description: This design features a serene and calming aesthetic, employing a monochromatic blue-gray color palette that evokes a sense of tranquility. The layout is structured and organized, with a notable emphasis on clean typography and spaciousness. Each section is clearly delineated, contributing to an overall coherent and harmonious visual experience, while the soft textures in the background enhance the serene mood.
Categories: minimalism, typography, print design, web interface, aesthetic design
Visual Characteristics: monochromatic palette, serene mood, structured layout, clean typography, spaciousness

Design 020:
Design 020:
Description: The design features a serene and professional layout with a muted color palette, emphasizing clean lines and minimalist aesthetics. It balances a centered, text-focused main content area with a sidebar offering functional navigation and supplementary information. The harmonious integration of serene imagery and subtle

Pick a model good for chat and tools

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o", 
    temperature=0,
    streaming=True
)

model.bind_tools(tool_belt)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1245518d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x124548e50>, root_client=<openai.OpenAI object at 0x1108f9310>, root_async_client=<openai.AsyncOpenAI object at 0x115d92090>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'design_retriever', 'description': 'Retrieves similar designs based on style requirements', 'parameters': {'properties': {'requirements': {'type': 'object'}, 'num_examples': {'default': 3, 'type': 'integer'}}, 'required': ['requirements'], 'type': 'object'}}}]}, config={}, config_factories=[])

Initialize state


In [12]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Set up the nodes and graph


In [16]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="""You are a helpful design assistant that can retrieve and analyze design examples. 
When a user describes their design preferences or requirements, use the design_retriever tool to find relevant examples.

Always use the design_retriever tool when:
- A user describes specific design requirements
- A user asks to see similar designs
- You need to find design inspiration based on user preferences

Format the requirements as a dictionary with these keys:
- style_description: Brief description of desired visual style
- key_elements: List of important visual elements
- color_scheme: Description of colors
- layout_preferences: Layout requirements
- mood: Desired emotional impact
""")

def call_model(state):
  messages = [system_message] + state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)
uncompiled_graph.set_entry_point("agent")


def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
  "agent",
  should_continue
)
uncompiled_graph.add_edge("action", "agent")

graph = uncompiled_graph.compile()

#formatted chain

def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

graph_chain = convert_inputs | graph | parse_output



Try it out!

In [ ]:
from langchain_core.messages import HumanMessage

async for chunk in graph.astream({"messages" : [HumanMessage(content="Hello, how are you?")]}, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content="Hello! I'm here and ready to help you with any design needs or questions you might have. How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-4edce0b5-fdec-4d5d-a4a6-92430faca51a-0')]





Let's see if the RAG tool works.

In [18]:
# Create a test message
from langchain_core.messages import HumanMessage

test_message = HumanMessage(
    content="""I want to see a design matching this description: 
    I want it to use a monochromatic color scheme with subtle accent colors. 
    The layout should be grid-based with clear hierarchy. 
    The overall mood should be professional and sophisticated."""
)

async for chunk in graph.astream({"messages" : [test_message]}, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content="To find a design that matches your description, I'll use the design_retriever tool. Here are the requirements based on your description:\n\n- style_description: Monochromatic with subtle accents\n- key_elements: Grid-based layout, clear hierarchy\n- color_scheme: Monochromatic with subtle accent colors\n- layout_preferences: Grid-based\n- mood: Professional and sophisticated\n\nLet's find some examples for you.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-8fa2e4af-671c-4c75-82fd-a7b3d6237e54-0')]



